<a href="https://colab.research.google.com/github/keisuke58/keisuke58/blob/main/Biofilm_20251112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os # Needed for saving files
from datetime import datetime # --- NEW IMPORT ---

# =============================================================================
# ## 1. 変数定義 (Variable Definitions)
# =============================================================================
print("Setting up parameters...")

# --- 相互作用係数 (a_ij) ---
a11 = 1.0
a12 = 5.0
a13 = 5.0
a14 = 5.0
a22 = 1.0
a23 = 3.0
a24 = 3.0
a33 = 1.0
a34 = 2.0
a44 = 1.0
A = np.array([
    [a11, a12, a13, a14],
    [a12, a22, a23, a24], # Assuming a21 = a12
    [a13, a23, a33, a34], # Assuming a31 = a13
    [a14, a24, a34, a44]  # Assuming a41 = a14
])

# --- 抗生物質感受性 (b_ii) ---
b11 = 0.4
b22 = 0.3
b33 = 0.2
b44 = 0.1
b_diag = np.array([b11, b22, b33, b44])

# --- 粘性 (Viscosities) ---
Eta1 = 0.8
Eta2 = 1.0
Eta3 = 1.5
Eta4 = 2.0
Eta_vec = np.array([Eta1, Eta2, Eta3, Eta4])
Eta_phi_vec = Eta_vec.copy()

# --- ペナルティ係数 (Penalty Parameter) ---
Kp1 = 1e-4

# --- ソルバーのパラメータ (Solver Parameters) ---
dt = 1e-6             # タイムステップ
maxtimestep = 150000    # ステップの総数
eps = 1e-6            # ニュートン法の許容誤差
one = 0.999           # (Mathematica 'one' variable)
tmax = maxtimestep * dt
tt = 0.0              # 現在の時間を初期化

# --- 栄養供給と抗生物質レベル (Nutrient & Antibiotic) ---
def c(t):
    return 50.0 + 50.0 * np.sin(500.0 * t)

def alpha(t):
    return 10.0

# =============================================================================
# ## 1.5 ファイル名とフォルダのセットアップ (Folder & Filename Setup)
# =============================================================================
#
# --- THIS IS THE NEW SECTION ---
#
# 1. ベースとなるファイル名を定義
base_filename = "AusgabeFall01-changing"

# 2. タイムスタンプを生成 (例: 20251112_160500)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 3. タイムスタンプ付きのフォルダ名を作成
folder_name = f"results_{timestamp}"

# 4. フォルダを作成
os.makedirs(folder_name, exist_ok=True)
print(f"Created results folder: {folder_name}")

# 5. すべてのファイルパスを、新しいフォルダとタイムスタンプを使うように更新
varDateiname = os.path.join(folder_name, f"{base_filename}_params_{timestamp}.txt")
dataDateiname = os.path.join(folder_name, f"{base_filename}_data_{timestamp}.dat")

# (プロットのファイル名もここで定義しておくとクリーンです)
plot_monitor1_name = os.path.join(folder_name, f"{base_filename}_monitor1_{timestamp}.png")
plot_monitor2_name = os.path.join(folder_name, f"{base_filename}_monitor2_{timestamp}.png")
plot_monitor3_name = os.path.join(folder_name, f"{base_filename}_monitor3_{timestamp}.png")
plot_iterations_name = os.path.join(folder_name, f"{base_filename}_iterations_{timestamp}.png")
plot_condition_name = os.path.join(folder_name, f"{base_filename}_conditioning_{timestamp}.png")

# =============================================================================
# ## 1.6 パラメータを保存 (Save Parameters)
# =============================================================================
# (このセクションは変更不要です。'varDateiname' を自動的に使います)
#
print(f"Saving parameters to {varDateiname}...")
ahat11 = a11/Eta1
ahat12 = a12/Eta1
ahat21 = a12/Eta2
ahat22 = a22/Eta2
# ... (all other ahat/a/b variables)

with open(varDateiname, "w") as f: # "w" = write (clobber old file)
    f.write(f"maxtimestep = {maxtimestep}\n")
    f.write(f"dt = {dt}\n")
    f.write(f"eps = {eps}\n")
    f.write(f"ahat11 = {ahat11}\n")
    f.write(f"ahat12 = {ahat12}\n")
    f.write(f"a11 = {a11}\n")
    f.write(f"a12 = {a12}\n")
    f.write(f"b11 = {b11}\n")
    f.write(f"b22 = {b22}\n")
    f.write(f"b33 = {b33}\n")
    f.write(f"b44 = {b44}\n")
    f.write(f"Eta1 = {Eta1}\n")
    f.write(f"Eta2 = {Eta2}\n")
    f.write(f"Eta3 = {Eta3}\n")
    f.write(f"Eta4 = {Eta4}\n")
    f.write(f"Kp1 = {Kp1}\n")
    f.write(f"c_function = 50 + 50 * Sin[500 t]\n")
    f.write(f"alpha_function = 10\n")

# =============================================================================
# ## 2. 初期条件とモニター (Initial Conditions & Monitors)
# =============================================================================
# (このセクションは変更ありません)
#
print("Initializing state vectors and monitors...")
Phi1i = 0.02
Phi2i = 0.02
Phi3i = 0.02
Phi4i = 0.02
Phi5i = 1.0 - (Phi1i + Phi2i + Phi3i + Phi4i)
g_prev = np.array([Phi1i, Phi2i, Phi3i, Phi4i, Phi5i, one, one, one, one, eps])
g_new_guess = np.array([eps, eps, eps, eps, one, one, one, one, one, 0.0])
monitor_phi1 = [Phi1i]
monitor_phi2 = [Phi2i]
monitor_phi3 = [Phi3i]
monitor_phi4 = [Phi4i]
monitor_phi0 = [Phi5i]
monitor_psi1 = [one]
monitor_psi2 = [one]
monitor_psi3 = [one]
monitor_psi4 = [one]
monitor_sum = [np.sum(g_prev[0:5])]
monitor_phipsi1 = [Phi1i * one]
monitor_phipsi2 = [Phi2i * one]
monitor_phipsi3 = [Phi3i * one]
monitor_phipsi4 = [Phi4i * one]
monitor_c = [c(0.0)]
monitor_alpha = [alpha(0.0)]
monitor_time = [0.0]
monitor_iterations = []
monitor_detK = []
monitor_conditioning = []

# =============================================================================
# ## 3. システム関数 (System Functions)
# =============================================================================
# (このセクションは変更ありません)

def compute_Q_vector(g_new, g_old, t, dt):
    """
    Mathematicaの「exp // Simplify」の出力を計算します。
    (*** psi のペナルティ項のバグを修正済み ***)
    """
    phi_new = g_new[0:4]
    phi0_new = g_new[4]
    psi_new = g_new[5:9]
    gamma_new = g_new[9]
    phi_old = g_old[0:4]
    phi0_old = g_old[4]
    psi_old = g_old[5:9]
    phidot = (phi_new - phi_old) / dt
    phi0dot = (phi0_new - phi0_old) / dt
    psidot = (psi_new - psi_old) / dt
    Q = np.zeros(10)
    CapitalPhi = phi_new * psi_new
    Interaction_dot_product = A @ CapitalPhi
    c_t_value = c(t)
    term1_phi = (Kp1 * (2.0 - 4.0 * phi_new)) / (np.power(phi_new - 1.0, 3) * np.power(phi_new, 3))
    term2_phi = (1.0 / Eta_vec) * (gamma_new + \
                 (Eta_phi_vec + Eta_vec * psi_new**2) * phidot + \
                 Eta_vec * phi_new * psi_new * psidot)
    term3_phi = (c_t_value / Eta_vec) * psi_new * Interaction_dot_product
    Q[0:4] = term1_phi + term2_phi - term3_phi
    Q[4] = gamma_new + \
           (Kp1 * (2.0 - 4.0 * phi0_new)) / (np.power(phi0_new - 1.0, 3) * np.power(phi0_new, 3)) + \
           phi0dot
    term1_psi = (-2.0 * Kp1) / (np.power(psi_new - 1.0, 2) * np.power(psi_new, 3)) - \
                (2.0 * Kp1) / (np.power(psi_new - 1.0, 3) * np.power(psi_new, 2))
    term2_psi = (b_diag * alpha(t) / Eta_vec) * psi_new
    term3_psi = phi_new * psi_new * phidot + phi_new**2 * psidot
    term4_psi = (c_t_value / Eta_vec) * phi_new * Interaction_dot_product
    Q[5:9] = term1_psi + term2_psi + term3_psi - term4_psi
    Q[9] = np.sum(phi_new) + phi0_new - 1.0
    return Q

# ---------------------------------------------------------------------------

def compute_Jacobian_matrix(g_new, g_old, t, dt):
    """
    CForm K の出力を実装します。
    (*** これは、すべての転写バグが修正された、
           完全に新しいヤコビアン関数です ***)
    """
    v = g_new
    phi_new = g_new[0:4]
    phi0_new = g_new[4]
    psi_new = g_new[5:9]
    phidot = (phi_new - g_old[0:4]) / dt
    psidot = (psi_new - g_old[5:9]) / dt
    c_t_value = c(t)
    CapitalPhi = phi_new * psi_new
    Interaction_dot_product = A @ CapitalPhi
    K = np.zeros((10, 10))
    phi_p_deriv = (Kp1*(-4. + 8.*v[0:4]))/(np.power(v[0:4],3)*np.power(v[0:4]-1.,3)) - \
                    (Kp1*(2. - 4.*v[0:4]))*(3./(np.power(v[0:4],4)*np.power(v[0:4]-1.,3)) + 3./(np.power(v[0:4],3)*np.power(v[0:4]-1.,4)))
    phi0_p_deriv = (Kp1*(-4. + 8.*v[4]))/(np.power(v[4],3)*np.power(v[4]-1.,3)) - \
                     (Kp1*(2. - 4.*v[4]))*(3./(np.power(v[4],4)*np.power(v[4]-1.,3)) + 3./(np.power(v[4],3)*np.power(v[4]-1.,4)))
    psi_p_deriv = (4.0 * Kp1 * (3.0 - 5.0*v[5:9] + 5.0*v[5:9]**2)) / (np.power(v[5:9], 4) * np.power(v[5:9] - 1.0, 4))
    for i in range(4):
        for j in range(4):
            K[i, j] = (c_t_value / Eta_vec[i]) * psi_new[i] * (-A[i, j] * psi_new[j])
        K[i, i] = phi_p_deriv[i] + \
                  (1.0 / Eta_vec[i]) * ( (Eta_phi_vec[i] + Eta_vec[i] * psi_new[i]**2) / dt + Eta_vec[i] * psi_new[i] * psidot[i] ) - \
                  (c_t_value / Eta_vec[i]) * ( psi_new[i] * (Interaction_dot_product[i] + A[i, i] * psi_new[i]) )
        K[i, 4] = 0.0
        for j in range(4):
            K[i, j+5] = (c_t_value / Eta_vec[i]) * psi_new[i] * (-A[i, j] * phi_new[j])
        K[i, i+5] = (1.0 / Eta_vec[i]) * ( 2.0 * Eta_vec[i] * psi_new[i] * phidot[i] + Eta_vec[i] * phi_new[i] * psidot[i] + Eta_vec[i] * phi_new[i] * psi_new[i] / dt ) - \
                      (c_t_value / Eta_vec[i]) * ( (Interaction_dot_product[i] + A[i, i] * phi_new[i] * psi_new[i]) + psi_new[i] * (A[i, i] * phi_new[i]) )
        K[i, 9] = 1.0 / Eta_vec[i]
    K[4, 4] = phi0_p_deriv + 1.0/dt
    K[4, 9] = 1.0
    for i in range(4):
        k = i + 5
        for j in range(4):
            K[k, j] = - (c_t_value / Eta_vec[i]) * ( A[i, j] * psi_new[j] * (phi_new[i]) + (Interaction_dot_product[i]) * (1.0 if i == j else 0.0) )
        K[k, i] = (psi_new[i] * phidot[i] + psi_new[i] * phi_new[i] / dt + 2.0 * phi_new[i] * psidot[i]) - \
                    (c_t_value / Eta_vec[i]) * ( A[i, i] * psi_new[i] * (phi_new[i]) + (Interaction_dot_product[i]) + phi_new[i] * (A[i, i] * psi_new[i]) )
        K[k, 4] = 0.0
        for j in range(4):
            K[k, j+5] = - (c_t_value / Eta_vec[i]) * phi_new[i] * (A[i, j] * phi_new[j])
        K[k, i+5] = psi_p_deriv[i] + \
                      (b_diag[i] * alpha(t) / Eta_vec[i]) + \
                      (phi_new[i] * phidot[i] + phi_new[i]**2 / dt) - \
                      (c_t_value / Eta_vec[i]) * phi_new[i] * (A[i, i] * phi_new[i])
        K[k, 9] = 0.0
    K[9, 0] = 1.0
    K[9, 1] = 1.0
    K[9, 2] = 1.0
    K[9, 3] = 1.0
    K[9, 4] = 1.0
    return K

# =============================================================================
# ## 4. ニュートン・ラプソン ソルバー (Newton-Raphson Solver)
# =============================================================================
# (このセクションは変更ありません)
#
print("Starting simulation...")
start_time = time.time()
np.seterr(divide='ignore', invalid='ignore')
K_matrix = np.zeros((10,10))

for step in range(maxtimestep):
    tt = (step + 1) * dt
    newton_iter = 0
    max_error = 1.0
    g_new = g_prev.copy()

    while (max_error > eps) and (newton_iter < 100):
        Q = compute_Q_vector(g_new, g_prev, tt, dt)
        K_matrix = compute_Jacobian_matrix(g_new, g_prev, tt, dt)

        if np.isnan(Q).any() or np.isnan(K_matrix).any():
            print(f"Error: NaN detected in Q or K at t={tt}")
            max_error = -2
            break

        try:
            dg = np.linalg.solve(K_matrix, -Q)
        except np.linalg.LinAlgError:
            print(f"Error: Jacobian matrix is singular at t={tt}")
            max_error = -1
            break

        g_new = g_new + dg
        max_error = np.max(np.abs(Q))
        newton_iter += 1

    if max_error == -1 or max_error == -2:
        print("Simulation stopped due to fatal error.")
        break
    # (We removed the warning for newton_iter >= 100)

    # --- モニターを更新 (Update Monitors) ---
    monitor_time.append(tt)
    monitor_phi1.append(g_new[0])
    monitor_phi2.append(g_new[1])
    monitor_phi3.append(g_new[2])
    monitor_phi4.append(g_new[3])
    monitor_phi0.append(g_new[4])
    monitor_psi1.append(g_new[5])
    monitor_psi2.append(g_new[6])
    monitor_psi3.append(g_new[7])
    monitor_psi4.append(g_new[8])
    monitor_sum.append(np.sum(g_new[0:5]))
    monitor_phipsi1.append(g_new[0] * g_new[5])
    monitor_phipsi2.append(g_new[1] * g_new[6])
    monitor_phipsi3.append(g_new[2] * g_new[7])
    monitor_phipsi4.append(g_new[3] * g_new[8])
    monitor_c.append(c(tt))
    monitor_alpha.append(alpha(tt))
    monitor_iterations.append(newton_iter)
    monitor_detK.append(np.linalg.det(K_matrix))
    monitor_conditioning.append(np.linalg.cond(K_matrix))

    g_prev = g_new.copy()

    if (step + 1) % 1000 == 0:
        print(f"Step {step+1}/{maxtimestep} (t = {tt:.4f}), Iterations: {newton_iter}, Max Error: {max_error:.2e}")

end_time = time.time()
print(f"Simulation finished in {end_time - start_time:.2f} seconds.")

# =============================================================================
# ## 5. 出力とプロット (Output and Plotting)
# =============================================================================
#
# --- THIS SECTION IS UPDATED TO SAVE ALL PLOTS ---
#
print("Plotting results...")

# --- Plot monitor1 (phi, psi, sum) ---
plt.figure(figsize=(10, 6))
plt.plot(monitor_time, monitor_phi1, label='phi_1', color='red', linestyle='-')
plt.plot(monitor_time, monitor_phi2, label='phi_2', color='blue', linestyle='-')
plt.plot(monitor_time, monitor_phi3, label='phi_3', color='yellow', linestyle='-')
plt.plot(monitor_time, monitor_phi4, label='phi_4', color='green', linestyle='-')
plt.plot(monitor_time, monitor_phi0, label='phi_0 (Empty)', color='black', linestyle='-')
plt.plot(monitor_time, monitor_psi1, label='psi_1', color='red', linestyle='--')
plt.plot(monitor_time, monitor_psi2, label='psi_2', color='blue', linestyle='--')
plt.plot(monitor_time, monitor_psi3, label='psi_3', color='yellow', linestyle='--')
plt.plot(monitor_time, monitor_psi4, label='psi_4', color='green', linestyle='--')
plt.plot(monitor_time, monitor_sum, label='Sum(phis)', color='orange', linestyle='-')
plt.title(f'All Variables (monitor1) - {timestamp}')
plt.xlabel('Time (t)')
plt.ylabel('Value')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.tight_layout()
plt.savefig(plot_monitor1_name, bbox_inches='tight')
print(f"Saved plot to {plot_monitor1_name}")
plt.show()

# --- Plot monitor2 (phi*psi) ---
plt.figure(figsize=(10, 6))
plt.plot(monitor_time, monitor_phipsi1, label='phi1*psi1', color='red')
plt.plot(monitor_time, monitor_phipsi2, label='phi2*psi2', color='blue')
plt.plot(monitor_time, monitor_phipsi3, label='phi3*psi3', color='yellow')
plt.plot(monitor_time, monitor_phipsi4, label='phi4*psi4', color='green')
plt.title(f'Interaction Variables (monitor2) - {timestamp}')
plt.xlabel('Time (t)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.savefig(plot_monitor2_name)
print(f"Saved plot to {plot_monitor2_name}")
plt.show()

# --- Plot monitor3 (c, alpha) ---
plt.figure(figsize=(10, 4))
plt.plot(monitor_time, monitor_c, label='c (Nutrient)', color='green')
plt.plot(monitor_time, monitor_alpha, label='alpha (Antibiotic)', color='red')
plt.title(f'Inputs (monitor3) - {timestamp}')
plt.xlabel('Time (t)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.savefig(plot_monitor3_name)
print(f"Saved plot to {plot_monitor3_name}")
plt.show()

# --- Plot cc (Iterations) ---
plt.figure(figsize=(10, 4))
plt.plot(monitor_iterations)
plt.title(f'Newton-Raphson Iterations per Time Step - {timestamp}')
plt.xlabel('Time Step')
plt.ylabel('Iterations')
plt.grid(True)
plt.savefig(plot_iterations_name)
print(f"Saved plot to {plot_iterations_name}")
plt.show()

# --- Plot myConditioning (Condition Number) ---
plt.figure(figsize=(10, 4))
plt.plot(monitor_conditioning)
plt.yscale('log')
plt.title(f'Jacobian Condition Number - {timestamp}')
plt.xlabel('Time Step')
plt.ylabel('Condition Number (Log Scale)')
plt.grid(True)
plt.savefig(plot_condition_name)
print(f"Saved plot to {plot_condition_name}")
plt.show()

# =============================================================================
# ## 6. 最終データのエクスポート (Final Data Export)
# =============================================================================
# (このセクションは変更不要です。'dataDateiname' を自動的に使います)
#
print(f"Exporting all data to {dataDateiname}...")

timestep_col = np.arange(len(monitor_time))
data_to_export = {
    "timestep": timestep_col,
    "phi1": monitor_phi1,
    "phi2": monitor_phi2,
    "phi3": monitor_phi3,
    "phi4": monitor_phi4,
    "phi0": monitor_phi0,
    "psi1": monitor_psi1,
    "psi2": monitor_psi2,
    "psi3": monitor_psi3,
    "psi4": monitor_psi4,
    "Sum(phis)": monitor_sum,
    "phi1psi1": monitor_phipsi1,
    "phi2psi2": monitor_phipsi2,
    "phi3psi3": monitor_phipsi3,
    "phi4psi4": monitor_phipsi4,
    "c": monitor_c,
    "alpha": monitor_alpha
}

df = pd.DataFrame(data_to_export)
df.to_csv(dataDateiname, sep="\t", index=False)

print(f"Done. All files saved to folder: {folder_name}")